In [28]:
# Домашнее задание
# На выбор
# * Создать MongoDB, записать туда данные из парсера с предыдущего урока (любое название базы, любое название таблицы). Выполнить команду для демонстрации содержимого коллекции. Прикрепить скриншот.
# * Создать базу данных sqlite, загрузить туда данные из парсера с предыдущего урока. Загрузить файл .db

In [62]:
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
import pprint
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [63]:
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

In [64]:
url = 'http://quotes.toscrape.com'
req = requests.get(url)
req.status_code

200

In [65]:
soup = bs(req.content, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Quotes to Scrape</title>
<link href="/static/bootstrap.min.css" rel="stylesheet"/>
<link href="/static/main.css" rel="stylesheet"/>
</head>
<body>
<div class="container">
<div class="row header-box">
<div class="col-md-8">
<h1>
<a href="/" style="text-decoration: none">Quotes to Scrape</a>
</h1>
</div>
<div class="col-md-4">
<p>
<a href="/login">Login</a>
</p>
</div>
</div>
<div class="row">
<div class="col-md-8">
<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="

In [66]:
def scrapeTeamPage(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    teamInfo = soup.find('div', 'profile_info')

In [67]:
import scrapy


class QuotesSpider(scrapy.Spider):
    name = 'quotes'
    allowed_domains = ['quotes.toscrape.com']
    start_urls = ['http://quotes.toscrape.com/']

    def parse(self, response):
        quotes = response.xpath("//*[@class='quote']")
        for quote in quotes:
            text = quote.xpath(".//*[@class='text']/text()").extract_first()
            author = quote.xpath(".//*[@class='author']/text()").extract_first()
            tags = quote.xpath(".//meta[@class='keywords']/@content").extract_first()

            yield{'Text':text,
                  'Author':author,
                  'Tags':tags}


        next_page_url = response.xpath("//*[@class='next']/a/@href").extract_first()
        absolute_next_page_url = response.urljoin(next_page_url)
        yield scrapy.Request(absolute_next_page_url)

In [72]:
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
import pprint
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient['quotes']
def connection_to_db():
    #Возвращение коннектора к базе данных Mongo
    quotes = db.quotes
    return db.quotes

def get_quotes(target):
    #Запрос и возвращение контента
    return requests.get(f'http://quotes.toscrape.com/={target}').text

def parse_quotes(content):
    #Парсит содержание
    
    items = []

    soup = bs(content, 'html.parser')
    divs = soup.find_all('div', 'quote')

for i in range(1,6):
    res = requests.get('http://quotes.toscrape.com/page/' + str(i))
    soup = bs(res.text, 'html.parser')

    quotes = soup.find_all('div', {'class':'quote'})
    print("Страница номер: " + str(i))
    print()
    for q in quotes:
        msg = q.find('span', {'class':'text'})
        auth = q.find('small')
        print(msg.text)
        print(auth.text)
        print()
        
def save_content_to_mongo(quotes):
    #Кладет контент в MongoDB
    db.quotes = connection_to_db()
    db.quotes.insert_many(items)

Страница номер: 1

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein

“It is our choices, Harry, that show what we truly are, far more than our abilities.”
J.K. Rowling

“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein

“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Jane Austen

“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Marilyn Monroe

“Try not to become a man of success. Rather become a man of value.”
Albert Einstein

“It is better to be hated for what you are than to be loved for what you are not.”
André Gide

“I have not failed. I've just found 10,000 ways that won't work.”
Thomas A. Edison

“A woman is like a tea bag; you never know how strong it is until it's in hot water.”
Ele

In [73]:
#Удаляет старый контент
db.quotes = connection_to_db()
db.quotes.drop()

#Сохраняет контент в базу
save_content_to_mongo(db.quotes)

NameError: name 'items' is not defined

In [74]:
def get_df_from_divs_extended(divs):
    
    quotes = []
    for div in divs:
        theme = div.find('a').text
        
        text = quote.xpath(".//*[@class='text']/text()").text
        author = quote.xpath(".//*[@class='author']/text()").extract_first()
        tags = quote.xpath(".//meta[@class='keywords']/@content").extract_first()
        
        quotes.append({
            'text': text,
            'author': author,
            'tags': tags
        })

    return pd.DataFrame(quotes)[['text', 'author', 'tags']].sort_values(by=['theme'])

In [76]:
df.to_csv(r'/dir/Parsing_task4.csv')

NameError: name 'df' is not defined